![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [2]:
import IPython
ip = IPython.get_ipython()
ip.run_line_magic("load_ext", "autoreload")


In [5]:
ip.run_line_magic("autoreload", "0")

In [5]:
# %load_ext autoreload
# %autoreload 0

In [22]:
# # 1.
import importlib
import research.condor_research as cr
import research.condor_fantasy as cf
importlib.reload(cr)
CondorResearch = cr.CondorResearch



importlib.reload(cf)
CondorFantasy = cf.CondorFantasy

import research.config as cfg
importlib.reload(cfg)
CondorResearchConfig = cfg.CondorResearchConfig


import research.trade_rules as tr
# from research.trade_rules import rules
importlib.reload(tr)
trade_rules = tr.rules

# import research.models.rule_set as rs
# import research.models.size_rule as sr
# import research.models.rule_condition as rc
# import research.models.trade_rule as tr
# importlib.reload(rs)
# importlib.reload(sr)
# importlib.reload(rc)
# importlib.reload(tr)
# RuleSet = rs.RuleSet
# SizeRule = sr.SizeRule
# RuleCondition = rc.RuleCondition
# TradeRule = tr.TradeRule

# 2. 
cfg = CondorResearchConfig()

# 3.
import json
import pandas as pd
qb = QuantBook()

key = 'ec279b1c2a070c50a299f2a66da4aaef_2024-10-02_00-00-00_trade_analytics.json'
snp_key = 'ec279b1c2a070c50a299f2a66da4aaef_2024-10-02_00-00-00_trade_snapshots.json'

CR = None
df = None
if qb.object_store.contains_key(key):
    string_data = qb.object_store.read(f"{key}")
    json_data = json.loads(string_data)    
    df = pd.json_normalize(json_data)
    CR = CondorResearch(df, cfg)


snp_df = None
if qb.object_store.contains_key(snp_key):
    string_data = qb.object_store.read(f"{snp_key}")
    json_data = json.loads(string_data)    
    snp_df = pd.json_normalize(json_data)
    

#     def __init__(self, df: pd.DataFrame, cr: CondorResearch, rules: list[dict], cfg=None):
CF = CondorFantasy(CR, trade_rules)

In [ ]:
print(CR.df.iloc[0].to_dict())

In [ ]:
row0 = CR.df.iloc[0].to_dict()

result = CF.evaluate_row(row0, trace=True)

print(f"ALLOWED={result['allowed']} SIZE={result['size_mult']}")
for r in result["rules"]:
    print(str(r))

In [23]:

results = []
for i in range(len(CF.df)):
    row = CF.df.iloc[i].to_dict()
    out = CF.evaluate_row(row, trace=False)  # fast
    results.append({
        "allowed": out["allowed"],
        "size_mult": out["size_mult"],  # 0 if not allowed
    })

res_df = pd.DataFrame(results, index=CF.df.index)
df2 = CF.df.join(res_df)

In [25]:
print("total rows og: ", len(CR.df))
print("total rows:", len(df2))
print("allowed rows:", int(df2["allowed"].sum()))


In [ ]:
v = CF.df["adx_bucket"].dropna().iloc[0]
print(repr(v))
print(repr(trade_rules[0]["skip"][0]["adx_bucket"]))

In [ ]:
df2["pnl_base"] = df2["pnl"]

# gated only (skip trades removed)
df2["pnl_gated"] = df2["pnl"].where(df2["allowed"], 0.0)

# gated + sized (skip removed + size applied)
df2["pnl_sized"] = df2["pnl"] * df2["size_mult"]


In [37]:
def summarize(name, s):
    total = float(s.sum())
    mean = float(s.mean())
    wins = int((s > 0).sum())
    losses = int((s < 0).sum())
    n = int((s != 0).sum())  # trades that count
    win_rate = (wins / (wins + losses)) if (wins + losses) else 0.0
    return {
        "name": name,
        "count": n,
        "total_pnl": total,
        "avg_pnl": mean,
        "win_rate": win_rate,
        "wins": wins,
        "losses": losses,
    }

base = summarize("baseline", df2["pnl_base"])
gated = summarize("gated_only", df2["pnl_gated"])
sized = summarize("gated_plus_sized", df2["pnl_sized"])

print(base)
print(gated)
print(sized)

In [35]:
skipped = df2.loc[~df2["allowed"]]

print("skipped trades:", len(skipped))
print("skipped total pnl:", float(skipped["pnl"].sum()))
print("skipped avg pnl:", float(skipped["pnl"].mean()))
print("skipped win rate:", float((skipped["pnl"] > 0).mean()))


In [36]:
print(df2["pnl_base"].quantile(0.01))
print(df2["pnl_sized"].quantile(0.01))


In [ ]:
df2["cum_pnl_base"] = df2["pnl_base"].cumsum()
df2["cum_pnl_gated"] = df2["pnl_gated"].cumsum()
df2["cum_pnl_sized"] = df2["pnl_sized"].cumsum()


In [41]:
CONTRACT_MULT = 100  # example

df2["pnl_base_$"] = df2["pnl_base"] * CONTRACT_MULT
df2["pnl_sized_$"] = df2["pnl_sized"] * CONTRACT_MULT

print(df2["pnl_base_$"].sum())
print(df2["pnl_sized_$"].sum())
